In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [5]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# List Parser

In [6]:
# Additional imports for list parser.
from langchain.output_parsers import CommaSeparatedListOutputParser

In [17]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)

# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()

print()

In [18]:
instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [21]:
# Define a query as a string, combining with the instructions.
query = "List out the presidents of the US." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
# Parse the result, store it, and print it.
result = llm.invoke(query)
print(result.content)

George Washington, John Adams, Thomas Jefferson, James Madison, James Monroe, John Quincy Adams, Andrew Jackson, Martin Van Buren, William Henry Harrison, John Tyler, James K. Polk, Zachary Taylor, Millard Fillmore, Franklin Pierce, James Buchanan, Abraham Lincoln, Andrew Johnson, Ulysses S. Grant, Rutherford B. Hayes, James A. Garfield, Chester A. Arthur, Grover Cleveland, Benjamin Harrison, William McKinley, Theodore Roosevelt, William Howard Taft, Woodrow Wilson, Warren G. Harding, Calvin Coolidge, Herbert Hoover, Franklin D. Roosevelt, Harry S. Truman, Dwight D. Eisenhower, John F. Kennedy, Lyndon B. Johnson, Richard Nixon, Gerald Ford, Jimmy Carter, Ronald Reagan, George H. W. Bush, Bill Clinton, George W. Bush, Barack Obama, Donald Trump, Joe Biden


In [22]:
data = parser.parse(result.content)
print(data)

['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Quincy Adams', 'Andrew Jackson', 'Martin Van Buren', 'William Henry Harrison', 'John Tyler', 'James K. Polk', 'Zachary Taylor', 'Millard Fillmore', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Andrew Johnson', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'James A. Garfield', 'Chester A. Arthur', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Howard Taft', 'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman', 'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon B. Johnson', 'Richard Nixon', 'Gerald Ford', 'Jimmy Carter', 'Ronald Reagan', 'George H. W. Bush', 'Bill Clinton', 'George W. Bush', 'Barack Obama', 'Donald Trump', 'Joe Biden']


# Structured Parser

In [24]:
# Additional imports for structured parser.
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [25]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)

# Define the schemas for our parser.
schemas = [
    ResponseSchema(name="country", description="the country"),
    ResponseSchema(name='capital', description="the capital")
]

In [26]:
schemas

[ResponseSchema(name='country', description='the country', type='string'),
 ResponseSchema(name='capital', description='the capital', type='string')]

In [29]:
# Initialize the output parser using the schema.
parser = StructuredOutputParser.from_response_schemas(schemas)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()

print(instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"country": string  // the country
	"capital": string  // the capital
}
```


In [32]:
# Define a query as a string, combining with the instructions.
query = "Name a country and its capital" + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)

print(result.content)

```json
{
	"country": "France",
	"capital": "Paris"
}
```


In [35]:
# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print()

{'country': 'France', 'capital': 'Paris'}



In [38]:
# Define a new query using the parsed output.
query = f"What are three tourist attractions {data['capital']}?"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)

In [40]:
print(result.content)

1. Eiffel Tower - An iconic symbol of Paris, the Eiffel Tower offers breathtaking views of the city and is a must-visit attraction for tourists.

2. Louvre Museum - One of the world's largest and most famous museums, the Louvre is home to thousands of works of art including the Mona Lisa and the Venus de Milo.

3. Notre-Dame Cathedral - A masterpiece of French Gothic architecture, Notre-Dame Cathedral is a historic and cultural landmark in Paris that is popular with tourists for its stunning architecture and rich history.


# Class-based Parser

In [41]:
# Additional imports for Pydantic parser.
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field, BaseModel

In [42]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)


# Define the class for our parsed responses.
class Country(BaseModel):
    name : str = Field(Description='the country name')
    capital : str = Field(Description='the capital name')
    population : str = Field(Description='the country population')

# Initialize the output parser using the schema.
parser = PydanticOutputParser(pydantic_object=Country)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()

print(instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "Description": "the country name", "type": "string"}, "capital": {"title": "Capital", "Description": "the capital name", "type": "string"}, "population": {"title": "Population", "Description": "the country population", "type": "string"}}, "required": ["name", "capital", "population"]}
```


In [43]:
# Define a query as a string, combining with the instructions.
query = "Name any country, its capital, and the country's population." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

{
  "name": "Japan",
  "capital": "Tokyo",
  "population": "126.5 million"
}


In [44]:
data = parser.parse(result.content)
print(data)
# Parse the result, store it, and print it.

name='Japan' capital='Tokyo' population='126.5 million'


In [45]:
# Define a new query using the parsed output.
query = f"What are three tourist attractions in {data.capital}?"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

1. Tokyo Disneyland and Tokyo DisneySea: These popular theme parks offer a wide range of rides, attractions, and entertainment for visitors of all ages.

2. Senso-ji Temple: Located in the historic Asakusa district, this ancient Buddhist temple is one of Tokyo's most famous landmarks and a popular tourist attraction.

3. Tsukiji Fish Market: This bustling seafood market is a must-visit for food lovers, offering a wide variety of fresh seafood, sushi, and other Japanese culinary delights.
